In [184]:
import numpy as np
import pandas as pd
import scipy.io
import scipy.constants as sc
from pygmid import Lookup as lk

#### Technology data

In [185]:
n = lk('../../techsweep/simulation/nfet_01v8_lvt.mat')
p = lk('../../techsweep/simulation/pfet_01v8.mat')
n_mat = scipy.io.loadmat('../../techsweep/simulation/nfet_01v8_lvt.mat', squeeze_me=True)
p_mat = scipy.io.loadmat('../../techsweep/simulation/pfet_01v8.mat', squeeze_me=True)

#### Specifications

In [186]:
T=300; A0=30; PM=80; NOI=50e-6

#### Design choices

In [187]:
# pick large gm/ID for input pairs, smaller gm/ID for mirror (to reduce noise)
gm_id = np.zeros(3)
gm_id[1] = 20
gm2_gm1 = 0.5
gm_id[2] = gm2_gm1*gm_id[1]
gm_id[0] = gm_id[1]

#### Sizing and benchmarking

In [188]:
# Find shortest channel length that satisfies gain requirement
# pfet model is bad for intrinsic gain, so just pick same L as for nfet
# add some margin to account for pfet gds (which is poorly modeled)
# A0 = gm[1]/(gds[1]+gds[2])
gm_gds1 = 2*A0
l_vec1 = n_mat['nfet_01v8_lvt']['L'].flatten()[0]
gm_gds_vec1 = n.lookup('GM_GDS', GM_ID=gm_id[1], L=l_vec1, VDS=0.3)
l_index1 = next(x for x, val in enumerate(gm_gds_vec1) if val > gm_gds1)
l = np.zeros(3)
l[1]=l_vec1[l_index1]
l[0]=l[1]
l[2]=l[1]

# calculate load capacitance for noise spec
# neglect device caps, consider only explicit cl
# overdesign a little to account for 1/f noise
# NOI = ( gamma[1]*(1+2*gamma[2]/gamma[1]) * sc.Boltzmann*T/cl)**0.5
gamma = np.zeros(3)
gamma[1] = n.lookup('STH_GM', GM_ID=gm_id[1], L=l[1])/(4*sc.Boltzmann*T)
gamma[2] = p.lookup('STH_GM', GM_ID=gm_id[2], L=l[2])/(4*sc.Boltzmann*T)
cl = 1.2*gamma[1]*(1+2*gamma[2]/gamma[1]) * sc.Boltzmann*T/NOI**2

# calculate mirror pole frequency and UGF that guarantees phase margin
# this calculation has some margin, as it neglects the LHP zero that returns some of the phase
# phi = np.arctan(UGF/fp2)*180/np.pi
fp2 = 0.5*p.lookup('GM_CGG', GM_ID=gm_id[2], L=l[2])/2/np.pi
phi = 90-PM
fp2_UGF = 1/np.tan(phi/180*np.pi)
UGF = fp2/fp2_UGF

# calculate gm1 based on UGF target
# neglect device caps, consider only explicit cl
# UGF = gm[1]/cl/2/np.pi
gm = np.zeros(3)
gm[1] = UGF*cl*2*np.pi

# calculate bias current and all widths
ib = 2*gm[1]/gm_id[1]
id = ib*np.array([1, 0.5, 0.5])
w = np.zeros(3)
w[0] = id[0]/n.lookup('ID_W', GM_ID=gm_id[0], L=l[0])
w[1] = id[1]/n.lookup('ID_W', GM_ID=gm_id[1], L=l[1])
w[2] = id[2]/p.lookup('ID_W', GM_ID=gm_id[2], L=l[2])
wfing = 5
nf = 1+np.floor_divide(w, wfing)

df = pd.DataFrame( [cl/1e-12, UGF/1e6, ib/1e-6], \
                   ['cl (pF)', 'UGF (MHz)', 'ib (uA)'], columns=['Value']); df.round(2)

,Value
cl (pF),7.02
UGF (MHz),83.32
ib (uA),367.36


In [189]:
df = pd.DataFrame( [w, l, nf, gamma], \
                   ['w (um)', 'l (um)', 'nf', 'gamma'], columns=['M0', 'M1', 'M2']); df.round(2)

,M0,M1,M2
w (um),485.62,242.81,96.40
l (um),0.60,0.60,0.60
nf,98.00,49.00,20.00
gamma,0.00,1.47,1.03


#### Write spice include file

In [190]:
# override with baseline values if desired
if False:
  ib = 10e-6; cl = 1e-12
  l = np.array([1.0, 0.5,  1.0])
  w = np.array([2.0, 20.0, 2.0])
  nf = np.array([2, 4, 2])

with open('sizing_ota-5t-buf.spice', 'w') as file:
    file.write(".param ib = " + "{:.2e}".format(ib) + '\n')
    file.write(".param cl = " + "{:.2e}".format(cl) + '\n')
    for i in range(len(l)):
      file.write(".param w%d = " % i + "{:.2f}".format(w[i]) + '\n')
      file.write(".param l%d = " % i + "{:.2f}".format(l[i]) + '\n')
      file.write(".param nf%d = " % i + "{:.2f}".format(nf[i]) + '\n')